In [233]:
from shapely.geometry import MultiLineString, LineString
from shapely.ops import unary_union
import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
def rnet_subset(rnet_x, rnet_y, dist=20, crop=True, min_length=0, rm_disconnected=True):
    # Calculate original length of each line in rnet_x
    rnet_x['length_x_original'] = rnet_x.geometry.length
    
    # Create a union of all geometries in rnet_y and create a buffer around it
    rnet_y_union = unary_union(rnet_y.geometry)
    rnet_y_buffer = rnet_y_union.buffer(dist, cap_style=3)
    
    # Convert the buffered geometry back to a GeoDataFrame
    rnet_y_buffer_gdf = gpd.GeoDataFrame(geometry=[rnet_y_buffer], crs=rnet_y.crs)
    
    if crop:
        # Perform intersection and update rnet_x
        rnet_x = gpd.overlay(rnet_x, rnet_y_buffer_gdf, how='intersection')
        # Cast MultiLineString to LineString
        rnet_x['geometry'] = rnet_x['geometry'].apply(line_cast)
        # Explode the geometry column to convert MultiLineStrings to LineStrings
        rnet_x = rnet_x.explode('geometry').reset_index(drop=True)
        if not isinstance(rnet_x, gpd.GeoDataFrame):
            rnet_x = gpd.GeoDataFrame(rnet_x, geometry='geometry')
    else:
        # Filter using within operation
        rnet_x = rnet_x[rnet_x.geometry.within(rnet_y_buffer)]
        
    # Filter lines shorter than min_length
    if min_length > 0:
        rnet_x = rnet_x[rnet_x['geometry'].apply(lambda geom: geom.length) > min_length]

    
    # Remove disconnected components if necessary
    if rm_disconnected:
        rnet_x = rnet_connected(rnet_x)
        
    return rnet_x


# def line_cast(geometry):
#     if isinstance(geometry, MultiLineString):
#         return [LineString(line) for line in geometry.geoms]
#     else:
#         return geometry

def line_cast(geometry):
    if isinstance(geometry, MultiLineString):
        return [LineString(line) for line in geometry.geoms]
    elif isinstance(geometry, LineString):
        return geometry
    else:
        raise TypeError(f"Unsupported geometry type: {type(geometry)}")

def rnet_connected(rnet):
    G = nx.Graph()

    # Add edges to the graph based on spatial relationships between line segments
    for idx1, row1 in rnet.iterrows():
        for idx2, row2 in rnet.iterrows():
            if idx1 != idx2 and row1['geometry'].intersects(row2['geometry']):
                G.add_edge(idx1, idx2)

    # Find the connected components in the graph
    connected_components = list(nx.connected_components(G))

    # Find the largest connected component
    largest_component = max(connected_components, key=len)

    # Create a new GeoDataFrame containing only the line segments in the largest connected component
    rnet_largest_group = rnet.loc[list(largest_component)]

    return rnet_largest_group


def rnet_merge(rnet_x, rnet_y, dist=5, funs=None, sum_flows=True, dist_subset=20, **kwargs):
    rnet_x.crs = rnet_y.crs
    if funs is None:
        funs = {}
        for col in rnet_y.columns:
            if pd.api.types.is_numeric_dtype(rnet_y[col]):
                funs[col] = np.sum

    sum_cols = [name for name, func in funs.items() if func == np.sum]
    rnetj = rnet_join(rnet_x, rnet_y, dist=dist, dist_subset=dist_subset, **kwargs)

    rnetj_df = rnetj.drop(columns=rnetj.geometry.name)

    # Define aggregation function
    def apply_funs(group, funs = funs):
        result = {}
        for col_name, fn in funs.items():
            nm = col_name
            if fn == np.sum and sum_flows:
                res = np.sum(group[nm] * group['length_y'])
            else:
                res = group[nm].apply(fn).sum()
            result[nm] = res
        result['corr_line_geometry'] = group['corr_line_geometry'].iloc[0]
        return pd.Series(result)

    res_df = rnetj_df.groupby(rnetj_df.columns[0]).apply(apply_funs, funs = funs).reset_index()
    # res_sf = rnet_x.join(res_df.set_index(rnetj_df.columns[0]), on=rnet_x.index)
    res_sf = rnet_x.reset_index().merge(res_df.reset_index(), left_on='index', right_on='index', how='left')

    if sum_flows:
        res_sf['length_x'] = res_sf.geometry.length
        length_y = rnet_y.geometry.length.sum()
        
        # Calculate the angle between 'corr_line_geometry' and 'geometry' for each row
        res_sf['angle'] = res_sf.apply(lambda row: calculate_angle(get_vector(gpd.GeoSeries(row['corr_line_geometry']).iloc[0]), 
                                                            get_vector(row['geometry'])), axis=1)
       
        for col_name in sum_cols:
            # Apply the adjustment only for rows where angle is less than 15 degrees
            mask = (res_sf['angle'] < 15) | (res_sf['angle'] > 160)
            res_sf.loc[mask, col_name] = res_sf.loc[mask, col_name] / res_sf.loc[mask, 'length_x']
            
            over_estimate = (res_sf.loc[mask, col_name] * res_sf.loc[mask, 'length_x']).sum() / (rnet_y[col_name] * length_y).sum()
            res_sf.loc[mask, col_name] = res_sf.loc[mask, col_name] / over_estimate

    return res_sf


def rnet_join(rnet_x, rnet_y, dist=10, length_y=True, key_column=1, subset_x=True, dist_subset=10, segment_length=0, endCapStyle="flat"):
    # Define the end cap style mapping

    cap_style_mapping = {"flat": 1, "round": 2, "square": 3}
    cap_style = cap_style_mapping.get(endCapStyle.lower(), 1)
    
    # If subset_x is true, subset rnet_x based on rnet_y
    if subset_x:
        rnet_x = rnet_subset(rnet_x, rnet_y, dist=dist_subset)
    
    # Apply buffer to rnet_x
    rnet_x_buffer = rnet_x.geometry.buffer(dist, cap_style=cap_style)
    rnet_x_buffer_geo_df = gpd.GeoDataFrame(geometry=rnet_x_buffer, crs=rnet_x.crs)
    
    
    # If the geometry column contains lists, explode it to create individual geometries
    if any(isinstance(geom, list) for geom in rnet_y['geometry']):
        rnet_y = rnet_y.explode('geometry')

    # If segment_length > 0, split rnet_y into segments (assuming line_segment function is defined)
    if segment_length > 0:
        # rnet_y = line_segment(rnet_y, segment_length=segment_length)
        rnet_y['geometry'] = rnet_y['geometry'].apply(lambda geom: line_segment(geom, segment_length=segment_length))
    # If length_y is true, calculate the length and assign to a new column
    if length_y:
        rnet_y['length_y'] = rnet_y['geometry'].apply(lambda geom: geom.length)
        
    # Calculate centroids of rnet_y
    rnet_y_centroids = rnet_y.copy()
    rnet_y_centroids['corr_line_geometry'] = rnet_y_centroids['geometry']
    rnet_y_centroids['geometry'] = rnet_y.geometry.centroid
    
    # Perform spatial join
    rnetj = gpd.sjoin(rnet_x_buffer_geo_df[[ 'geometry']], rnet_y_centroids, how='inner', op='intersects')

    return rnetj


def line_segment(l, segment_length=20):
    # If geometry is a LineString, treat it as a single-item list
    if isinstance(l, LineString):
        lines = [l]
    elif isinstance(l, MultiLineString):
        lines = l.geoms
    else:
        print("Unexpected geometry type:", type(l))
        print("Geometry:", l)
        return l  # Return the original geometry

    result_geoms = []
    for line in lines:
        # Calculate the number of segments
        l_length = line.length
        n_segments = max(round(l_length / segment_length), 1)
        if n_segments == 1:
            result_geoms.append(line)
            continue

        # Create segments
        from_to_sequence = [i/n_segments for i in range(n_segments + 1)]
        line_segment_list = [line.interpolate(from_to_sequence[i], normalized=True).coords for i in range(n_segments)]
        line_segment_list = [LineString([line_segment_list[i][0], line_segment_list[i + 1][0]]) for i in range(n_segments - 1)]
        result_geoms.extend(line_segment_list)

    # Create a MultiLineString and return
    return MultiLineString(result_geoms)

def get_vector(line):
    if isinstance(line, LineString):
        start, end = line.coords[:2]
    else:  # for MultiLineStrings, just use the first line
        start, end = line.geoms[0].coords[:2]
    return [end[0] - start[0], end[1] - start[1]]

# Calculate angle between two vectors 
def calculate_angle(vector1, vector2):
    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
    magnitude_product = math.sqrt(vector1[0]**2 + vector1[1]**2) * math.sqrt(vector2[0]**2 + vector2[1]**2)
    cos_angle = dot_product / magnitude_product
    angle = math.degrees(math.acos(cos_angle))
    return angle

In [234]:
rnet_x_url = "https://github.com/ropensci/stplanr/releases/download/v1.0.2/rnet_x_ed.geojson"
rnet_y_url = "https://github.com/ropensci/stplanr/releases/download/v1.0.2/rnet_y_ed.geojson"

rnet_x = gpd.read_file(rnet_x_url)
rnet_x = rnet_x.to_crs(epsg=3857) 
rnet_y = gpd.read_file(rnet_y_url)
rnet_y = rnet_y.to_crs(epsg=3857) 

total_distance_traveled = round(sum(rnet_y['value'] * rnet_y['geometry'].length))
print(total_distance_traveled/1000)

# fig, ax = plt.subplots(figsize=(12, 10))
# rnet_y.plot(column='value', ax=ax, legend=True, cmap='Blues')
# ax.set_title("res_sf by value")
# plt.show()


30613.666


In [247]:
print(rnet_x.shape)
rnet_x = rnet_subset(rnet_x, rnet_y, dist = 40)
print(rnet_x.shape)
rnet_x = rnet_subset(rnet_x, rnet_y, dist = 40, min_length = 10)
print(rnet_x.shape)

(474, 4)


c:\Users\wangz\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


(466, 4)


C:\Users\wangz\AppData\Local\Temp\ipykernel_22044\2302198713.py:22: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:3857

  rnet_x = gpd.overlay(rnet_x, rnet_y_buffer_gdf, how='intersection')
c:\Users\wangz\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


(452, 4)


In [235]:
funs = {'value': np.sum, 'Quietness': np.mean}
rnet_merged = rnet_merge(rnet_x, rnet_y, dist=20, segment_length=20, funs=funs)

c:\Users\wangz\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
C:\Users\wangz\AppData\Local\Temp\ipykernel_22044\4056242055.py:2: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  rnet_merged = rnet_merge(rnet_x, rnet_y, dist=20, segment_length=20, funs=funs)
C:\Users\wangz\AppData\Local\Temp\ipykernel_22044\2302198713.py:162: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:3857

  rnetj = gpd.sjoin(rnet_x_buffer_geo_df[[ 'geometry']], rnet_y_centroids, how='inner', op='intersects')


In [244]:
total_distance_traveled = round(sum(rnet_merged['value'] * rnet_merged['geometry'].length))
print(total_distance_traveled/1000)

2750.413


In [249]:
input_list = [
    'all_fastest_bicycle', 'all_fastest_bicycle_ebike',
    'all_fastest_bicycle_go_dutch', 'all_quietest_bicycle',
    'all_quietest_bicycle_ebike', 'all_quietest_bicycle_go_dutch',
    'commute_fastest_bicycle', 'commute_fastest_bicycle_ebike',
    'commute_fastest_bicycle_go_dutch', 'commute_quietest_bicycle',
    'commute_quietest_bicycle_ebike', 'commute_quietest_bicycle_go_dutch',
    'primary_fastest_bicycle', 'primary_fastest_bicycle_ebike',
    'primary_fastest_bicycle_go_dutch', 'primary_quietest_bicycle',
    'primary_quietest_bicycle_ebike', 'primary_quietest_bicycle_go_dutch',
    'secondary_fastest_bicycle', 'secondary_fastest_bicycle_ebike',
    'secondary_fastest_bicycle_go_dutch', 'secondary_quietest_bicycle',
    'secondary_quietest_bicycle_ebike',
    'secondary_quietest_bicycle_go_dutch', 'Gradient', 'Quietness'
]
